# NextLineDataset as Tensorflow Data

(C) Maxim Gansert, 2020, Mindscan Engineering

In [ ]:
import os
import json

import tensorflow as tf

In [ ]:
context_dataset = tf.data.TextLineDataset( 
    os.path.join('D:\\Downloads\\Big-Code-excerpt','NextLineTranslationDataset.jsonl.from'))
nextline_dataset = tf.data.TextLineDataset(
    os.path.join('D:\\Downloads\\Big-Code-excerpt','NextLineTranslationDataset.jsonl.to'))

In [ ]:
def my_json_decode(source, target):
    source_decoded = json.loads(source.numpy())
    target_decoded = json.loads(target.numpy())
    return source_decoded, target_decoded

def tf_context_nextline_json_decode(context, nextline):
    result_context, result_nextline = tf.py_function(my_json_decode, [context, nextline], [tf.int64, tf.int64])
    result_context.set_shape([None])
    result_nextline.set_shape([None])

    return result_context, result_nextline

# Length of a "sentence"
MAX_LENGTH = 96

# ??
BUFFER_SIZE = 50000

# Number of sentences processed in one batch
BATCH_SIZE = 96

def filter_max_length(x, y, max_length=MAX_LENGTH):
    return tf.logical_and(tf.size(x) <= max_length,
                        tf.size(y) <= max_length)

# combine both datasets - parallel corpus
train_dataset_ = tf.data.Dataset.zip((context_dataset, nextline_dataset))
# transform from string to bpe encoded message
train_dataset__ = train_dataset_.map(tf_context_nextline_json_decode)

# filter dataset entries exceeding the capacity
train_dataset = train_dataset__.filter(filter_max_length)

# now do preprocessing and shuffle data around
train_dataset = train_dataset.cache()
train_dataset = train_dataset.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE)
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)

# train_dataset should now be ready for the given transformer.
# We have to evaluate that whether everything worked ok

In [ ]:
for (batch, (context, nextline)) in enumerate(train_dataset):
    print('Batch: '+str(batch) + '\n')
    print('Context:\n')
    print(context)
    print('Context:\n')
    print(nextline)
    
    # only one round...
    break